In [142]:
# load data
using DelimitedFiles, LinearAlgebra
dist = readdlm("data42.txt",Int)

#dist = [[0 1 2 3]
#        [1 0 4 5]
#        [2 4 0 6]
#        [3 5 6 0]]

V = size(dist)[1]
E = (V-1)*(V) ÷ 2

ind = zeros(Int, (V,V)) # construct matrix with 2 Floyd's triangle
c = zeros(Int, E) # construct cost vector
for i = 2:V
    for j = 1:i-1
        ind_ij = (i-2)*(i-1) ÷ 2 + j
        ind[i,j] = ind_ij
        c[ind_ij] = dist[i,j] # Lazy Caterer's sequence + (j-1)
    end
end

ind = ind + transpose(ind)

42×42 Array{Int64,2}:
   0    1    2    4    7   11   16   22  …  596  631  667  704  742  781  821
   1    0    3    5    8   12   17   23     597  632  668  705  743  782  822
   2    3    0    6    9   13   18   24     598  633  669  706  744  783  823
   4    5    6    0   10   14   19   25     599  634  670  707  745  784  824
   7    8    9   10    0   15   20   26     600  635  671  708  746  785  825
  11   12   13   14   15    0   21   27  …  601  636  672  709  747  786  826
  16   17   18   19   20   21    0   28     602  637  673  710  748  787  827
  22   23   24   25   26   27   28    0     603  638  674  711  749  788  828
  29   30   31   32   33   34   35   36     604  639  675  712  750  789  829
  37   38   39   40   41   42   43   44     605  640  676  713  751  790  830
  46   47   48   49   50   51   52   53  …  606  641  677  714  752  791  831
  56   57   58   59   60   61   62   63     607  642  678  715  753  792  832
  67   68   69   70   71   72   73   74   

In [151]:
using Gurobi, JuMP
model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)

# define variables and constraints for the initial LP problem
@variable(model,0 <= x[1:E] <= 1)
@objective(model,Min,sum(c.*x))
for i = 1:V 
    indices = filter!(x -> x ≠ 0,ind[:,i])
    @constraint(model,sum(x[indices]) == 2)
end

optimize!(model)
#print(JuMP.value.(x))
JuMP.value.(x)[findall(x->(x ≠ 0.0 && x ≠ 1.0), JuMP.value.(x))]


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 42 rows, 861 columns and 1722 nonzeros
Model fingerprint: 0xd2c837b6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   8.100000e+01   0.000000e+00      0s
      55    6.4100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 55 iterations and 0.00 seconds
Optimal objective  6.410000000e+02


6-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5